# Calculate Data for the Surroundings of each zip code.

## 0. Initialize

In [223]:
#Libraries and Settings
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',10)

In [224]:
#Data File to Dataframe
file_1='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/2_Merge_Census.csv'
file_2='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/2_Merge_Irs.csv'
file_3='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/1_Load_Walmart.csv'
file_4='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/1_Load_Costco.csv'
file_5='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/1_Load_Geographic_Data.csv'

census=pd.read_csv(file_1, converters={'zip': str})
irs=pd.read_csv(file_2, converters={'zip': str})
walmart=pd.read_csv(file_3, converters={'zip': str})
costco=pd.read_csv(file_4, converters={'zip': str})
geo=pd.read_csv(file_5, converters={'zip': str})

print('census: ',census.shape)
print('irs: ', irs.shape)
print('walmart: ',walmart.shape)
print('costco: ',costco.shape)
print('geo: ', geo.shape)

census:  (33971, 127)
irs:  (27935, 28)
walmart:  (4654, 9)
costco:  (604, 5)
geo:  (33129, 12)


## 1. Get List of Zip Codes
It is necessary to perform the calculations only for those zip codes where there is a store and not for every single existing zip code. 

In [225]:
zip_codes = set(walmart['zip']).union(set(costco['zip']))
len(zip_codes) #4472 This is smaller than the total of stores, meaning there are more than one walmart and costo on the same zip code

#Are there any zip codes that are not present in the master?
master_zip_codes=set(geo['zip'])
print(zip_codes-master_zip_codes)


set()


## 2. Calculate the Appropriate Aggregate functions for each metric

### 2.1 Aggregate Metrics for Census

In [226]:
#I am going to use these empty dataframes to add in the calculation for each zip code using a loop
agg_census=pd.DataFrame()
agg_irs=pd.DataFrame()
agg_geo=pd.DataFrame()

for zip_i in zip_codes:
    #this brings the list of surounding zips for that particular zip_i 
    surr_zip = geo[geo['zip']==zip_i]['zips_around'].values[0]
    surr_zip=surr_zip.replace('[','').replace(']','').replace('\'','').split(', ')

    #keep only the ones in the list in this temporary df
    t_census=pd.DataFrame()
    t_census=census[census['zip'].isin(surr_zip)]
    t_census=t_census.dropna()

    #This will contain the calculated data for zip_i and will be added when finished with it.
    t_agg_census={'zip':zip_i}
    #now let's obtain the new data for zip_i

    columns=t_census.columns.tolist()
    for c in (columns[1:]):
        if ('delta' not in c)and('ratio' not in c):
            t_agg_census[c]=t_census[c].sum()
        if ('ratio' in c)and('delta' not in c):
            c_name=c.replace('_ratio','')
            if('21' in c):
                t_agg_census[c]=t_agg_census[c_name]/t_agg_census['total_pop_21']
            if('11' in c):
                t_agg_census[c]=t_agg_census[c_name]/t_agg_census['total_pop_11']
        if ('ratio' not in c)and('delta' in c):
            c_name=c.replace('delta_','')
            t_agg_census[c]=(t_agg_census[c_name+'_21'])-(t_agg_census[c_name+'_11'])
        if ('ratio' in c)and('delta' in c):
            c_name=c.replace('_ratio','').replace('delta_','')
            t_agg_census[c]=(t_agg_census[c_name+'_21']-t_agg_census[c_name+'_11'])/t_agg_census['total_pop_11']

    agg_census=pd.concat([agg_census,pd.DataFrame(t_agg_census, index=[0])])         
    #agg_census=pd.concat([agg_census,t_agg_census], ignore_index=True)         

In [230]:
#Check for nulls
sum(agg_census.isna().sum())

0

## 3.Export to CSV and verify in Excel some calculations

In [228]:
#Export to CSV and verify in Excel some calculations
'''
import os
outname = 'DELETE_t_agg_census.csv'
outdir = '/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)    

t_agg_census.to_csv(fullname, header=True, index=False)
print("Saved!")

import os
outname = 'DELETE_t_census.csv'
outdir = '/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)    

t_census.to_csv(fullname, header=True, index=False)
print("Saved!")
'''


'\nimport os\noutname = \'DELETE_t_agg_census.csv\'\noutdir = \'/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data\'\nif not os.path.exists(outdir):\n    os.mkdir(outdir)\nfullname = os.path.join(outdir, outname)    \n\nt_agg_census.to_csv(fullname, header=True, index=False)\nprint("Saved!")\n\nimport os\noutname = \'DELETE_t_census.csv\'\noutdir = \'/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data\'\nif not os.path.exists(outdir):\n    os.mkdir(outdir)\nfullname = os.path.join(outdir, outname)    \n\nt_census.to_csv(fullname, header=True, index=False)\nprint("Saved!")\n'